In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
pd.options.display.max_columns = None
df=pd.read_csv(r"../input/weather-dataset-rattle-package/weatherAUS.csv")
df

In [ ]:
df.isnull().sum()

In [ ]:
def fillAllValues(groupByColumn):
    categoricalColumns=['WindGustDir','WindDir9am','WindDir3pm','RainToday','RainTomorrow','Location']+[groupByColumn]
    gb=df.groupby(['Location',groupByColumn],as_index=False)
    def fillWithMode(column):
        mode=column.mode()
        if len(mode.index)>0:
            return column.fillna(mode.iloc[0])
        return column
    return pd.concat([gb[categoricalColumns].transform(fillWithMode),gb[df.columns.difference(categoricalColumns)].transform(lambda x:x.fillna(x.mean()))],axis=1)
df['MonthYear']=pd.DatetimeIndex(df.Date).to_period('M').astype(str)
df.drop(columns=['Date'],inplace=True)
df=fillAllValues('MonthYear')
df['Month']=pd.DatetimeIndex(df.MonthYear).month.astype(str)
df.drop(columns=['MonthYear'],inplace=True)
df=fillAllValues('Month')
df.dropna(axis=1,inplace=True)

In [ ]:
df=pd.get_dummies(df[['WindDir9am', 'WindDir3pm', 'RainToday', 'RainTomorrow', 'Location']],drop_first=True,prefix=['WindDir9am', 'WindDir3pm', 'RainToday', 'RainTomorrow', 'Location'])

In [ ]:
df1=df.drop(columns=['RainTomorrow_Yes'])
df1.columns

# Rain Today

In [ ]:
def TrainTestSplit(df,outputVariable,stratifySampling=False,transformationY=np.array,test_size=0.25):
    from sklearn.model_selection import train_test_split
    y=transformationY(df[outputVariable])
    return train_test_split(df[df.columns.difference([outputVariable])],y,random_state=0,stratify=y if stratifySampling else None,test_size=test_size)

In [ ]:
X_train,X_test,y_train,y_test=TrainTestSplit(df1,'RainToday_Yes',True)

In [ ]:
def ClassificationAlgorithms():
     
    class color:
        PURPLE = '\033[95m'
        CYAN = '\033[96m'
        DARKCYAN = '\033[36m'
        BLUE = '\033[94m'
        GREEN = '\033[92m'
        YELLOW = '\033[93m'
        RED = '\033[91m'
        BOLD = '\033[1m'
        UNDERLINE = '\033[4m'
        END = '\033[0m'
        
    from sklearn.metrics import confusion_matrix,accuracy_score
    #logistic Regression
    from sklearn.linear_model import LogisticRegression
    print(color.PURPLE + color.BOLD + color.UNDERLINE + 'LOGISTIC REGRESSION\n' + color.END)
    model=LogisticRegression()
    model.fit(X_train,y_train)
    pred=model.predict(X_test)
    print(f"Accuracy For Test Data : {accuracy_score(y_test,pred)}\n")
    print(f"Confusion Matrix For Test Data : \n{confusion_matrix(y_test, pred)}\n")
    pred=model.predict(X_train)
    print(f"\nAccuracy For Train Data : {accuracy_score(y_train,pred)}\n")
    print(f"Confusion Matrix For Train Data : \n{confusion_matrix(y_train, pred)}\n")
    #Decision Tree Classification
    print(color.CYAN + color.BOLD + color.UNDERLINE + '\n\n\nDECISION TREE CLASSIFICATION\n' + color.END)
    from sklearn.tree import DecisionTreeClassifier
    model=DecisionTreeClassifier()
    model.fit(X_train,y_train)
    pred=model.predict(X_test)
    print(f"Accuracy For Test Data : {accuracy_score(y_test,pred)}\n")
    print(f"Confusion Matrix For Test Data : \n{confusion_matrix(y_test, pred)}\n")
    pred=model.predict(X_train)
    print(f"\nAccuracy For Train Data : {accuracy_score(y_train,pred)}\n")
    print(f"Confusion Matrix For Train Data : \n{confusion_matrix(y_train, pred)}\n")
    #Random Forest Classification
    print(color.CYAN + color.BOLD + color.UNDERLINE + '\n\n\nRANDOM FOREST CLASSIFICATION\n' + color.END)
    from sklearn.ensemble import RandomForestClassifier
    model=RandomForestClassifier()
    model.fit(X_train,y_train)
    pred=model.predict(X_test)
    print(f"Accuracy For Test Data : {accuracy_score(y_test,pred)}\n")
    print(f"Confusion Matrix For Test Data : \n{confusion_matrix(y_test, pred)}\n")
    pred=model.predict(X_train)
    print(f"\nAccuracy For Train Data : {accuracy_score(y_train,pred)}\n")
    print(f"Confusion Matrix For Train Data : \n{confusion_matrix(y_train, pred)}\n")
    #StandardScaler
    from sklearn.preprocessing import StandardScaler
    ss=StandardScaler()
    ssTrain=ss.fit_transform(X_train)
    ssTest=ss.transform(X_test)
    #Support Vector Classification
    print(color.CYAN + color.BOLD + color.UNDERLINE + '\n\n\nSUPPORT VECTOR CLASSIFICATION\n' + color.END)
    from sklearn.ensemble import RandomForestClassifier
    model=RandomForestClassifier()
    model.fit(ssTrain,y_train)
    pred=model.predict(ssTest)
    print(f"Accuracy For Test Data : {accuracy_score(y_test,pred)}\n")
    print(f"Confusion Matrix For Test Data : \n{confusion_matrix(y_test, pred)}\n")
    pred=model.predict(ssTrain)
    print(f"\nAccuracy For Train Data : {accuracy_score(y_train,pred)}\n")
    print(f"Confusion Matrix For Train Data : \n{confusion_matrix(y_train, pred)}\n")
#     #KNN
#     print(color.CYAN + color.BOLD + color.UNDERLINE + '\n\n\nK NEAREST NEIGHBOUR\n' + color.END)
#     from sklearn.neighbors import KNeighborsClassifier
#     model=KNeighborsClassifier()
#     model.fit(ssTrain,y_train)
#     pred=model.predict(ssTest)
#     print(f"Accuracy For Test Data : {accuracy_score(y_test,pred)}\n")
#     print(f"Confusion Matrix For Test Data : \n{confusion_matrix(y_test, pred)}\n")
#     pred=model.predict(ssTrain)
#     print(f"\nAccuracy For Train Data : {accuracy_score(y_train,pred)}\n")
#     print(f"Confusion Matrix For Train Data : \n{confusion_matrix(y_train, pred)}\n")

In [ ]:
from sklearn.linear_model import LogisticRegression
def getBestThreshold(steps=.1,log=LogisticRegression()):  
    log.fit(X_train,y_train)
    testPredictedProbabilitiesOfOnes=log.predict_proba(X_test)[:,1]
    trainPredictedProbabilitiesOfOnes=log.predict_proba(X_train)[:,1]
    from sklearn.metrics import accuracy_score
    threshold=np.arange(0,1,steps)
    testResults=[]
    trainResults=[]
    thresholdPredictionBinary={}
    for i in threshold:
        thresholdPredictionBinary[i]=np.where(testPredictedProbabilitiesOfOnes>i,1,0)
        testResults.append(accuracy_score(y_test,thresholdPredictionBinary[i]))
        trainResults.append(accuracy_score(y_train,np.where(trainPredictedProbabilitiesOfOnes>i,1,0)))
    result=pd.DataFrame({'testResults':testResults,'trainResults':trainResults},threshold)
    result.plot(kind='line',xlabel="Thresholds",ylabel="Accuracy Score",figsize=(20,5))
    bestThreshold=result.testResults.idxmax()
    print(f"Best Accuracy For Test Results : {result.testResults.max()}\nBest Threshold For Test Results : {bestThreshold}\nAccuracy For Training Data With {bestThreshold} As Threshold : {result.trainResults.max()}")
    return testPredictedProbabilitiesOfOnes

In [ ]:
def plot_ROC_curve(pred):
    from sklearn.metrics import roc_curve,auc
    fpr, tpr, threshold = roc_curve(y_test, pred)
    roc_auc = auc(fpr, tpr)
    from matplotlib import pyplot as plt
    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()

In [ ]:
ClassificationAlgorithms()

In [ ]:
pred=getBestThreshold()

In [ ]:
plot_ROC_curve(pred)

# Rain Tomorrow

In [ ]:
X_train,X_test,y_train,y_test=TrainTestSplit(df,'RainTomorrow_Yes',True)

In [ ]:
ClassificationAlgorithms()

In [ ]:
pred=getBestThreshold()

In [ ]:
plot_ROC_curve(pred)